## import libraries 

In [1]:
import re
import requests
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from time import sleep
import time
from bs4 import BeautifulSoup
from datetime import datetime

### scraping redddit, and since the information is checked and deleted if wrong I will consider them all as true

In [2]:
def scrape_subreddit(subreddit,n):
    dataframes = []
    today = datetime.now()
    now = today.replace(hour=0, minute=0, second=0, microsecond=0)
    time = int(now.timestamp())
    for num in range(n):
        res = requests.get(f'https://api.pushshift.io/reddit/search/submission/?subreddit={subreddit}&size=100&before={time}')
        df = pd.DataFrame(res.json()['data'])
        dataframes.append(df)
        time = res.json()['data'][-1]['created_utc']
        sleep(5)
        print(f'Scraping {n} batches.')
    all_posts = pd.concat(dataframes)
    all_posts.to_csv('./data/subreddit.csv', index=False)
    return all_posts

In [3]:
coronavirus = scrape_subreddit('coronavirus', 25)

Scraping 25 out of 10 batches.
Scraping 25 out of 10 batches.
Scraping 25 out of 10 batches.
Scraping 25 out of 10 batches.
Scraping 25 out of 10 batches.
Scraping 25 out of 10 batches.
Scraping 25 out of 10 batches.
Scraping 25 out of 10 batches.
Scraping 25 out of 10 batches.
Scraping 25 out of 10 batches.
Scraping 25 out of 10 batches.
Scraping 25 out of 10 batches.
Scraping 25 out of 10 batches.
Scraping 25 out of 10 batches.
Scraping 25 out of 10 batches.
Scraping 25 out of 10 batches.
Scraping 25 out of 10 batches.
Scraping 25 out of 10 batches.
Scraping 25 out of 10 batches.
Scraping 25 out of 10 batches.
Scraping 25 out of 10 batches.
Scraping 25 out of 10 batches.
Scraping 25 out of 10 batches.
Scraping 25 out of 10 batches.
Scraping 25 out of 10 batches.


In [4]:
coronavirus.shape

(2500, 78)

dropping every column, except for title, since I do not need anything else.

In [5]:
coronavirus_df = coronavirus['title']
coronavirus_df = pd.DataFrame(coronavirus_df)

In [6]:
coronavirus_df

,title
0,US coronavirus vaccine trial on hold after UK ...
1,Penn State Reports 320 Cases Of Coronavirus On...
2,Chinese mainland reports 10 new COVID-19 cases...
3,Mask resistance during a pandemic isn't new – ...
4,U.S. adults experienced increased mental healt...
...,...
95,Niños y coronavirus: por qué una alta carga vi...
96,Coronavirus may dice heart muscle fibers into ...
97,Some will never learn and still go out before ...
98,Some people will never learn and still go out ...


as mentioned before, assuming all information on reddit is true

In [7]:
coronavirus_df['label'] = 'true'
coronavirus_df.rename(columns = {'title':'text'}, inplace=True)
coronavirus_df.to_csv('./data/coronavirus_subreddit.csv')

### importing and merging a  datat set from  [Kaggle](https://www.kaggle.com/ambityga/covid19misinformation) User Ambesh Shekhar, who had flase and misleading information gathered from ponyter with my reddits

In [8]:
poynter = pd.read_csv('./data/poynter_data.csv')
poynter

,label,text
0,misleading,A post that has a list of recommendations to ...
1,false,The first person to received a Chinese COVID-...
2,false,A video from family doctor Natalia Prego Canc...
3,false,RNA-based COVID-19 vaccines can turn humans i...
4,false,A post that shows a doctor from Santiago del ...
...,...,...
7539,false,The coronavirus was created in a lab and pate...
7540,misleading,A Chinese market caused the new coronavirus (...
7541,misleading,The peak of the new coronavirus will happen i...
7542,false,Stores and supermarkets in Veracruz (Mexico) ...


In [9]:
coronavirus_df.reset_index(inplace=True)
coronavirus_df = coronavirus_df[['label', 'text']]

In [10]:
df = pd.concat([coronavirus_df, poynter ])

#Formatting dataframe
df.reset_index(inplace=True)
df.drop(columns='index', inplace=True)
df.drop_duplicates(keep = 'first', inplace=True) 
df.to_csv('./data/combined_data.csv', index=False)

In [11]:
df.head()

,label,text
0,true,US coronavirus vaccine trial on hold after UK ...
1,true,Penn State Reports 320 Cases Of Coronavirus On...
2,true,Chinese mainland reports 10 new COVID-19 cases...
3,true,Mask resistance during a pandemic isn't new – ...
4,true,U.S. adults experienced increased mental healt...


### adding information from WHO as well

In [12]:
url = 'https://www.who.int/emergencies/diseases/novel-coronavirus-2019/advice-for-public/myth-busters'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'lxml')
h2s =soup.find_all('h2')

trues = []
for each in h2s[1:]:
    li_true = each.text[6:]
    trues.append(li_true)
    
who_data = pd.DataFrame({'label': 'true','text':[values for values in trues]})
combined_with_who = pd.concat([df, who_data],ignore_index=True)
combined_with_who.to_csv('./data/combined_with_who.csv', index = False)

In [13]:
combined_with_who

,label,text
0,true,US coronavirus vaccine trial on hold after UK ...
1,true,Penn State Reports 320 Cases Of Coronavirus On...
2,true,Chinese mainland reports 10 new COVID-19 cases...
3,true,Mask resistance during a pandemic isn't new – ...
4,true,U.S. adults experienced increased mental healt...
...,...,...
9969,true,Rinsing your nose with saline does NOT prevent...
9970,true,Eating garlic does NOT prevent COVID-19
9971,true,People of all ages can be infected by the COVI...
9972,true,Antibiotics CANNOT prevent or treat COVID-19


In [14]:
combined_with_who['label'].value_counts()

false         6315
true          2500
misleading    1159
Name: label, dtype: int64

Alright, this is all of my data, and the missleading vs true, and false information